#### Finding the Gregorian calendar date of the Lunar New Year in the year 4989

Why?

The visual novel [Hate Plus](https://store.steampowered.com/app/239700/Hate_Plus/) takes place in the year 4989. An important theme of the story (simplifying quite a bit here) is Korean culture, and in-game the new year occurs on February 10 (as evidenced by [this screenshot](https://lparchive.org/Hate-Plus/Update%2031/3-525.jpg) and the one below).

When I was playing through the game, a character mentioned that the new year is "tomorrow", which surprised me since that was the actual date of the new year IRL... but it was January.


![hate plus screenshot showing the predicted lunar new year date on feb 10](in_game_date.jpg "Title")

This made me wonder what the actual date of the new year in 4989 would be. Chinese new year [typically occurs during the second or third new moon after the winter solstice](https://en.wikipedia.org/wiki/Lunar_New_Year) so it turns out that this problem is strongly related to predicting the phases of the moon far into the future.

Many online tools for looking up the date of the lunar new year don't support dates beyond a couple hundred years into the future. Moreover, multiple LLMs (even after enabling chain-of-thought ("reasoning") and "deep research" modes, and using external tools) provided different answers. For example:

ChatGPT with "deep research" after 13 minutes of thinking: February 11, 4989

ChatGPT with "reasoning" mode after 22 seconds of thinking: February 17, 4989

ChatGPT with a "vanilla" prompt (without COT, the model wrote a simple Python script without external libraries): February 22, 4989

Gemini: "\[...] *it's impossible to definitively say when the Lunar New Year would fall in the year 4989.*"

Claude: "*Based on current astronomical models and calendrical calculations, the Lunar New Year in 4989 would likely fall in late January or early February, following the pattern we see today. However, I cannot provide the exact date with certainty given the astronomical variables involved over such an extended time period.*"

### Using the Skyfield package to try to find the answer

We'll need to use a long-range [ephemeris](https://rhodesmill.org/skyfield/planets.html) to find the predicted positions of astronomical bodies so far into the future.

NASA Jet Propulsion Laboratory's `de421.bsp` supports only dates from 1899-07-29 to 2053-10-09. However, `de441.bsp` contains data for the time range between 13,200 BCE and 17,191 CE, making it perfect for our use case.

The filesize is 3.1GB.

In [5]:
from skyfield.api import load

planets = load('de441.bsp')

[#################################] 100% de441.bsp


Testing that the ephemeris and Skyfield function correctly:

In [7]:
earth, mars = planets['mercury'], planets['venus']

ts = load.timescale()
t = ts.now()
position = earth.at(t).observe(mars)
ra, dec, distance = position.radec()

print(ra)
print(dec)
print(distance)

14h 57m 42.29s
-12deg 42' 11.8"
0.676358 au


Setting up the time range of interest for the new moons:

In [8]:
t_start = ts.utc(4988, 12, 1)
t_end = ts.utc(4989, 3, 1)

Finding the new moons ([phase 0](https://rhodesmill.org/skyfield/almanac.html#phases-of-the-moon)):

In [9]:
from skyfield.almanac import find_discrete, moon_phases, seasons

f = moon_phases(planets)
times, phases = find_discrete(t_start, t_end, f)

new_moons = [t for t, phase in zip(times, phases) if phase == 0]

Finding the winter solstice ([event type 3](https://rhodesmill.org/skyfield/almanac.html#the-seasons)):

In [11]:
season_func = seasons(planets)
solstice_times, solstice_events = find_discrete(t_start, t_end, season_func)

dec_solstice = [t for t, e in zip(solstice_times, solstice_events) if e == 3][0]

#### Second new moon after the solstice:

In [14]:
moons_after_solstice = [t for t in new_moons if t.tt > dec_solstice.tt]
second_new_moon = moons_after_solstice[1]
print(second_new_moon.utc_strftime())

4989-02-10 21:36:32 UTC


All the new moons in this time range:

In [15]:
for new_moon in moons_after_solstice:
    print(new_moon.utc_strftime())

4989-01-12 08:51:05 UTC
4989-02-10 21:36:32 UTC


### Therefore, according to this calculation, the Lunar New Year will occur on February 10, 4989.

Which is exactly the same date as shown in Hate Plus.

#### Additional remarks

* Since the predicted datetime is 4989-02-10 21:36:32 UTC, this would be 11th of February in the time zones of most Asian countries. This potentially means that the answer provided by ChatGPT with deep research mode enabled was actually correct. It's a bit hard to tell if that was just a fluke.
* Since this date is almost 3000 years into the future, there is some uncertainty involved. The ephemeris used (DE441) makes different assumptions than the one used for smaller timescales (DE440) so that it can support dates further away. There may be doubts about using predictions that are so far into the future - nevertheless, I believe that using these newest available tables published by NASA JPL constitutes the "best guess" we may currently have. For further information you can view the article introducing these ephemerides: https://ssd.jpl.nasa.gov/doc/Park.2021.AJ.DE440.pdf